In [3]:
import os
import pickle

# 폴더 경로 설정
folder_path = '/home/minwook0008/please/OR2/or2_case_study/current_crawling/naver-finance-news-crawler/data'
all_data = []
# 폴더 내의 모든 파일을 탐색
for filename in os.listdir(folder_path):
    # 피클 파일만 선택
    if filename.endswith('.pkl'):
        file_path = os.path.join(folder_path, filename)
        
        # 피클 파일 열기
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            for dt in data:
                all_data.append(dt)
            print(f'Contents of {filename}:')
            print(data)
            print('----------------------------------')


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
import json
output_file = 'news_data.jsonl'

# jsonl 파일에 기록
with open(output_file, 'w', encoding='utf-8') as f:
    for item in all_data:
        json_line = json.dumps(item, ensure_ascii=False)
        f.write(json_line + '\n')


In [9]:
!pip install easygoogletranslate

In [10]:
all_data

[{'press': '파이낸셜뉴스',
  'date': '2023-05-07',
  'time': '18:29:07',
  'title': "SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]",
  'link': 'https://finance.naver.com/news/news_read.naver?article_id=0005008095&office_id=014&mode=mainnews&type=&date=2023-05-07&page=1',
  'text': '',
  'cleaned_text': []},
 {'press': '파이낸셜뉴스',
  'date': '2023-05-07',
  'time': '11:38:03',
  'title': "'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]",
  'link': 'https://finance.naver.com/news/news_read.naver?article_id=0005007913&office_id=014&mode=mainnews&type=&date=2023-05-07&page=1',
  'text': '',
  'cleaned_text': []},
 {'press': '아이뉴스24',
  'date': '2023-05-07',
  'time': '10:01:04',
  'title': '[이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망',
  'link': 'https://finance.naver.com/news/news_read.naver?article_id=0000744845&office_id=031&mode=mainnews&type=&date=2023-05-07&page=1',
  'text': '',
  'cleaned_text': []},
 {'press': '이데일리',
  'date': '2023-05-07',
  'time': '09:19:01',
  'title': '[펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF',
  'link'

In [14]:
import multiprocessing
from easygoogletranslate import EasyGoogleTranslate
from functools import partial
from tqdm import tqdm
import json

def translate_title(data, translator):
    title = data['title']
    translated_title = translator.translate(title)
    data['english_title'] = translated_title
    return data

def worker(data_chunk, translator):
    results = []
    for data in data_chunk:
        result = translate_title(data, translator)
        results.append(result)
    return results

def main(data_list, num_workers):
    translator = EasyGoogleTranslate(
        source_language='ko',
        target_language='en',
        timeout=10
    )

    chunk_size = len(data_list) // num_workers
    data_chunks = [data_list[i:i + chunk_size] for i in range(0, len(data_list), chunk_size)]
    
    with multiprocessing.Pool(processes=num_workers) as pool:
        worker_with_translator = partial(worker, translator=translator)
        
        results = []
        for result in tqdm(pool.imap(worker_with_translator, data_chunks), total=len(data_chunks), desc="Translating"):
            results.extend(result)

    with open('output_trans.jsonl', 'w', encoding='utf-8') as f:
        for data in results:
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

    for data in results:
        print(f"Original: {data['title']}")
        print(f"Translated: {data['english_title']}\n")

if __name__ == '__main__':

    num_workers =32
    main(all_data, num_workers)

Translating:   0%|   | 0/33 [00:59<?, ?it/s]Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:



KeyboardInterrupt: 

In [ ]:
import multiprocessing
from EasyGoogleTranslate import EasyGoogleTranslate
from functools import partial

# 번역 함수
def translate_title(data, translator):
    title = data['title']
    translated_title = translator.translate(title)
    data['translated_title'] = translated_title
    return data

# 멀티프로세스 작업자 함수
def worker(data_chunk, translator):
    results = []
    for data in data_chunk:
        result = translate_title(data, translator)
        results.append(result)
    return results

def main(data_list, num_workers):
    translator = EasyGoogleTranslate(
        source_language='ko',
        target_language='en',
        timeout=10
    )

    chunk_size = len(data_list) // num_workers
    data_chunks = [data_list[i:i + chunk_size] for i in range(0, len(data_list), chunk_size)]

    with multiprocessing.Pool(processes=num_workers) as pool:
        worker_with_translator = partial(worker, translator=translator)
        
        results = pool.map(worker_with_translator, data_chunks)

    translated_data = [item for sublist in results for item in sublist]

    for data in translated_data:
        print(f"Original: {data['title']}")
        print(f"Translated: {data['translated_title']}\n")

if __name__ == '__main__':

    num_workers = 32

    main(data_list, num_workers)

In [19]:
import pandas as pd
import json
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    df = pd.DataFrame(data)
    return df

# 예시 사용법
file_path = '/home/minwook0008/please/OR2/or2_case_study/finbert_finetuning/news_data.jsonl'
df = load_jsonl_to_dataframe(file_path)
print(df)


        press        date      time  \
0      파이낸셜뉴스  2023-05-07  18:29:07   
1      파이낸셜뉴스  2023-05-07  11:38:03   
2      아이뉴스24  2023-05-07  10:01:04   
3        이데일리  2023-05-07  09:19:01   
4        이데일리  2023-05-07  09:13:01   
...       ...         ...       ...   
22595    이데일리  2023-01-02  16:06:02   
22596    이데일리  2023-01-02  15:52:01   
22597   머니투데이  2023-01-02  11:37:18   
22598   아시아경제  2023-01-02  08:27:40   
22599    이데일리  2023-01-02  08:13:01   

                                           title  \
0      SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]   
1         'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]   
2                 [이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망   
3                  [펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF   
4               美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망]   
...                                          ...   
22595  새해 첫날 환율, 저가 매수에 1272.6원…5거래일 만에 상승[외환마감]   
22596              [코스닥 마감] 결국 1% 하락…2차전지주 6% 강세   
22597     韓시장은 열렸지만…日·中 등 아시아 증시 줄줄이 휴장 [Asia오전]   
22598      

In [12]:
df.head()

,press,date,time,title,link,text,cleaned_text
0,파이낸셜뉴스,2023-05-07,18:29:07,SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망],https://finance.naver.com/news/news_read.naver...,,[]
1,파이낸셜뉴스,2023-05-07,11:38:03,'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망],https://finance.naver.com/news/news_read.naver...,,[]
2,아이뉴스24,2023-05-07,10:01:04,[이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망,https://finance.naver.com/news/news_read.naver...,,[]
3,이데일리,2023-05-07,09:19:01,[펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF,https://finance.naver.com/news/news_read.naver...,,[]
4,이데일리,2023-05-07,09:13:01,美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망],https://finance.naver.com/news/news_read.naver...,,[]


In [13]:
df = df.drop(columns=['link','text','cleaned_text','time','press'])

In [14]:
df.head()

,date,title
0,2023-05-07,SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]
1,2023-05-07,'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]
2,2023-05-07,[이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망
3,2023-05-07,[펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF
4,2023-05-07,美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망]


In [20]:
result_df = df.groupby('date').head(5)

# 결과 출력
print(result_df)

        press        date      time  \
0      파이낸셜뉴스  2023-05-07  18:29:07   
1      파이낸셜뉴스  2023-05-07  11:38:03   
2      아이뉴스24  2023-05-07  10:01:04   
3        이데일리  2023-05-07  09:19:01   
4        이데일리  2023-05-07  09:13:01   
...       ...         ...       ...   
22592    조선비즈  2023-01-02  16:31:01   
22593     뉴스1  2023-01-02  16:22:33   
22594     뉴시스  2023-01-02  16:07:49   
22595    이데일리  2023-01-02  16:06:02   
22596    이데일리  2023-01-02  15:52:01   

                                               title  \
0          SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]   
1             'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]   
2                     [이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망   
3                      [펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF   
4                   美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망]   
...                                              ...   
22592  [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환   
22593                  [시황종합] 코스피 개장일 0.48% 내린 2225선   
22594                  [마감시

In [2]:

import os
import sys
import urllib.request
client_id = "pyq4fq6nd8"
client_secret = "xmySeb2gSKckt6v4ERpUIKw51KqJDQ3xiMBQv1Em"
encText = urllib.parse.quote("번역할 문장을 입력하세요")
data = "source=ko&target=en&text=" + encText
url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"message":{"result":{"srcLangType":"ko","tarLangType":"en","translatedText":"Enter the sentence you want to translate"}}}


In [10]:
response_body.decode('utf-8')

'{"message":{"result":{"srcLangType":"ko","tarLangType":"en","translatedText":"Enter the sentence you want to translate"}}}'

In [14]:
import json
dictionary = json.loads(response_body.decode('utf-8'))

In [18]:
dictionary['message']['result']['translatedText']

'Enter the sentence you want to translate'

In [21]:

def save_dataframe_to_jsonl(result_df, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for index, row in result_df.iterrows():
            file.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

# 예시 사용법
output_file_path = 'samling_news_file.jsonl'
save_dataframe_to_jsonl(result_df, output_file_path)

print(f"Data saved to {output_file_path}")

Data saved to samling_news_file.jsonl


In [22]:
import pandas as pd
import json
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    df = pd.DataFrame(data)
    return df

# 예시 사용법
file_path = '/home/minwook0008/please/OR2/or2_case_study/finbert_finetuning/dataset/output_trans.jsonl'
df = load_jsonl_to_dataframe(file_path)
print(df)


       press        date      time  \
0     파이낸셜뉴스  2023-05-07  18:29:07   
1     파이낸셜뉴스  2023-05-07  11:38:03   
2     아이뉴스24  2023-05-07  10:01:04   
3       이데일리  2023-05-07  09:19:01   
4       이데일리  2023-05-07  09:13:01   
...      ...         ...       ...   
1648    조선비즈  2023-01-02  16:31:01   
1649     뉴스1  2023-01-02  16:22:33   
1650     뉴시스  2023-01-02  16:07:49   
1651    이데일리  2023-01-02  16:06:02   
1652    이데일리  2023-01-02  15:52:01   

                                              title  \
0         SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]   
1            'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]   
2                    [이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망   
3                     [펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF   
4                  美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망]   
...                                             ...   
1648  [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환   
1649                  [시황종합] 코스피 개장일 0.48% 내린 2225선   
1650                  [마감시황]코스피, 새해 첫날 기관 매물에 0.

In [23]:
result_df = df.groupby('date').head(5)

# 결과 출력
print(result_df)

       press        date      time  \
0     파이낸셜뉴스  2023-05-07  18:29:07   
1     파이낸셜뉴스  2023-05-07  11:38:03   
2     아이뉴스24  2023-05-07  10:01:04   
3       이데일리  2023-05-07  09:19:01   
4       이데일리  2023-05-07  09:13:01   
...      ...         ...       ...   
1648    조선비즈  2023-01-02  16:31:01   
1649     뉴스1  2023-01-02  16:22:33   
1650     뉴시스  2023-01-02  16:07:49   
1651    이데일리  2023-01-02  16:06:02   
1652    이데일리  2023-01-02  15:52:01   

                                              title  \
0         SG發 폭락 한숨 돌렸지만 美 지역은행 '위기설' 변수 [주간 증시 전망]   
1            'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망]   
2                    [이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망   
3                     [펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF   
4                  美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망]   
...                                             ...   
1648  [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환   
1649                  [시황종합] 코스피 개장일 0.48% 내린 2225선   
1650                  [마감시황]코스피, 새해 첫날 기관 매물에 0.

In [ ]:
result_df.to_csv('trans_output.csv', index=False)